In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import tool
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.agent import AgentFinish
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from pydantic import BaseModel, Field
from llama_index.core import SimpleDirectoryReader
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
import requests
import gradio as gr
import os


In [38]:
openai_llm = ChatOpenAI(temperature=0, model="gpt-4o")

In [39]:
cv_template = """
You are an expert in analyzing CVs and providing personalized career advice. Given an individual's CV, you will:

1. Conduct a detailed analysis of their skills, experience, qualifications, and overall suitability for various roles in the Malawian job market.
2. Recommend the best job opportunities that align with their profile, ensuring that the suggestions are well-matched to their strengths and career goals.
3. Take into account current industry trends, job market demand, and emerging opportunities in Malawi, particularly on myjobo (the job posting platform), to ensure that your recommendations are relevant and forward-thinking.
4. Provide actionable insights that can help improve the CV, such as highlighting areas for growth or skill enhancement based on market needs in Malawi.

Your goal is to provide comprehensive, data-driven advice that helps individuals make informed career decisions.

Below is the CV:
{input}

The output should be in a Python dict format with the following keys:

- "Skills_and_experience_analysis": A detailed analysis of the individual's skills and experience, including strengths, potential growth areas, and relevant qualifications.
- "job_opportunities": A list of **simple, clear job titles** that are relevant to the individual’s experience and align with job listings on myjobo 
in Malawi. Each title should be easily searchable on the platform. Examples: "Software Engineer", "Data Analyst", "Nurse", "Project Coordinator", "Teacher", "Administrator", "Graphic Designer", "Sales Representative",
 "Customer Service Officer", "Web Developer", "IT Support", "Accountant", "Driver", "Health Worker", etc.

**Important:** Ensure that the job titles are simple, commonly understood, and easy to search for on myjobo and other 
job posting platforms in Malawi. 
Avoid using long descriptions or overly specific roles. 
The titles should be relevant to the Malawian job market, and your analysis should ensure that the recommendations fit roles 
commonly listed on myjobo.

"""

email_template = """
You are an expert in writing application letters. Your task is to create a compelling and professional cover letter based on an individual's skills and experience, showcasing how they align with the job they're applying for.

Please ensure that the application letter includes:
- A strong introduction highlighting the candidate’s background and motivation for applying.
- A section detailing the key skills and experience that make the candidate a strong fit for the position.
- A concluding paragraph that emphasizes the candidate’s enthusiasm and readiness for the role.

Use a professional tone, and ensure the letter is clear and focused on presenting the candidate in the best possible light.

The candidate's CV:
{input}

The job being applied for is:
{job_description}

Please format the letter as follows:
1. Introduction
2. Skills and Experience
3. Closing and Enthusiasm
"""




In [45]:
def search_for_jobs(title, status):
    url = f"https://myjobo.com/search-jobs?keywords={title.lower().strip().replace(' ','+')}"
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, "lxml")
    job_results = soup.find_all("div", class_="job-card")
    job_details = []
    job_404 = soup.find("div", class_="col-md-12 text-center text-gray")
    if job_404:
        if 'No job vacancies match your search criteria.' in job_404.text.strip():
            job_details.append({'job_404':f"no vacancy found for {title} position"})
            status.Info(f"no vacancy found for {title} position")
            print(f"no vacancy found for {title} position")
            return job_details
    if len(job_results) > 0:
        for job in job_results:
            job_div = job.find('div', class_="card-body p-0")
            job_title = ""
            if job_div:
                job_title_tag = job_div.find("h5", class_="card-title text-secondary fs-18 mb-0")
                if job_title_tag:
                    job_title = job_title_tag.text.strip()
            
                job_div_loc = job_div.find_all("div", class_="desc d-flex me-4")
                job_location = ""
                for i in job_div_loc:
                    info = i.find("p", class_="fs-14 text-gray mb-2")
                    if info:
                        job_location = info.text.strip()
                post_date_div = job.find("div", class_="desc d-flex")
                post_date = ""
                if post_date_div:
                    post_date_p = post_date_div.find("p", class_="fs-14 text-gray mb-2")
                    if post_date_p:
                        post_date = post_date_p.text.strip()
                
                job_descprition_link_div = job.find("div", class_="mb-40")
                job_desc_link = ""
                if job_descprition_link_div:
                    job_desc_link_a = job_descprition_link_div.find("a", class_="card py-30 border-0")
                    if job_desc_link_a and 'href' in job_desc_link_a.attrs:
                        job_desc_link = job_desc_link_a['href'].strip()
                
               
                job_description_html = requests.get(job_desc_link).text
                desc_soup = BeautifulSoup(job_description_html, "lxml")
                
                
                job_description_div = desc_soup.find("div", class_="job-card")
                responsibilities_div = job_description_div.find("div", class_="key-responsibilities")
                responsibilities_list = []
                if responsibilities_div:
                    responsibilities = responsibilities_div.find_all("p")
                    for resp in responsibilities:
                        responsibilities_list.append(resp.text.strip())
                
                job_details.append({
                    'job_title': job_title,
                    'job_location': job_location,
                    'post_date': post_date,
                    'job_desc_link': job_desc_link,
                    'responsibilities': responsibilities_list
                })
                
    return job_details

In [46]:
def getting_job_opportunities(status):
    import json
    job_opportunities = []
    cleaned_response_string = cv_llm_response.strip('```python\n').strip('```')
    json_response = json.loads(cleaned_response_string)

    if json_response['job_opportunities'] is not None:
        for idx,job in enumerate(json_response['job_opportunities']):
            status.Info(f"searching for {job} position")
            print(f"searching for {job} position")
            response = search_for_jobs(job, status)
            for idx, job_response in enumerate(response):
                for job in job_response:
                    if 'job_404' not in job:
                        job_opportunities.append({'job_id':idx, "job_opportunity":job_response})
    return job_opportunities

In [57]:
def get_file(file_path, status):
    directory_name = os.path.dirname(file_path)  
    
    documents = SimpleDirectoryReader(directory_name).load_data()    
    prompt = ChatPromptTemplate.from_template(cv_template)
    chain = LLMChain(prompt=prompt, llm=openai_llm)
    global doc_info 
    doc_info = documents[0].text
    global cv_llm_response
    cv_llm_response = chain.run({"input":doc_info})
    job_oppo = getting_job_opportunities(status)
    structured_output = ""
    if len(job_oppo)> 0:
        for idx, job in enumerate(job_oppo):
            structured_output = job.get('responsibilities', [])
    return structured_output

In [58]:
def start_gradio():
    with gr.Blocks() as demo:
        status = gr
        gr.Markdown("# Upload your CV to search for job opportunities")
        output = gr.Text(label="Results:")
        file_input = gr.File(label="Upload your CV here in PDF format")
        btn = gr.Button("Send")
        @btn.click(inputs=file_input, outputs=[output])
        def call_get_file(input):
            return get_file(input, status)
    demo.launch()

In [59]:
start_gradio()

* Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.


searching for Software Engineer position
no vacancy found for Software Engineer position
searching for Web Developer position
no vacancy found for Web Developer position
searching for AI Engineer position
no vacancy found for AI Engineer position
searching for Data Analyst position
searching for IT Support position
no vacancy found for IT Support position
searching for Software Engineer position
no vacancy found for Software Engineer position
searching for Web Developer position
no vacancy found for Web Developer position
searching for AI Engineer position
no vacancy found for AI Engineer position
searching for Data Analyst position
searching for IT Support position
no vacancy found for IT Support position
searching for Digital Trainer position
no vacancy found for Digital Trainer position
searching for Software Engineer position
no vacancy found for Software Engineer position
searching for Web Developer position
no vacancy found for Web Developer position
searching for AI Engineer pos